# Clone Bert and Wongnai Dataset Gits

# Install Library

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!git clone https://github.com/ThAIKeras/bert

Cloning into 'bert'...
remote: Enumerating objects: 275, done.
remote: Total 275 (delta 0), reused 0 (delta 0), pack-reused 275
Receiving objects: 100% (275/275), 201.44 KiB | 5.44 MiB/s, done.
Resolving deltas: 100% (151/151), done.


# Import Dataset

In [ ]:
!git clone https://github.com/wongnai/wongnai-corpus.git

Cloning into 'wongnai-corpus'...
remote: Enumerating objects: 127, done.
remote: Total 127 (delta 0), reused 0 (delta 0), pack-reused 127
Receiving objects: 100% (127/127), 39.65 MiB | 40.32 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
%cd wongnai-corpus/review/

/content/wongnai-corpus/review


In [ ]:
!unzip review_dataset.zip

Archive:  review_dataset.zip
 extracting: sample_submission.csv   
  inflating: test_file.csv           
  inflating: w_review_train.csv      
   creating: __MACOSX/
  inflating: __MACOSX/._sample_submission.csv  
  inflating: __MACOSX/._test_file.csv  
  inflating: __MACOSX/._w_review_train.csv  


In [ ]:
%cd ../..

/content


# Download Pre-train Model and SPM files

In [ ]:
# download p-train thai keras
!gdown --id 1J3uuXZr_Se_XIFHj7zlTJ-C9wzI9W_ot

Downloading...
From: https://drive.google.com/uc?id=1J3uuXZr_Se_XIFHj7zlTJ-C9wzI9W_ot
To: /content/bert_base_th.zip
1.17GB [00:05, 228MB/s]


In [ ]:
# cut thai word
!gdown --id 1F7pCgt3vPlarI9RxKtOZUrC_67KMNQ1W

Downloading...
From: https://drive.google.com/uc?id=1F7pCgt3vPlarI9RxKtOZUrC_67KMNQ1W
To: /content/th_wiki_bpe.zip
100% 560k/560k [00:00<00:00, 36.3MB/s]


In [ ]:
# pretrain check point and cut word
!unzip bert_base_th.zip
!unzip th_wiki_bpe.zip

Archive:  bert_base_th.zip
   creating: bert_base_th/
  inflating: bert_base_th/model.ckpt.index  
  inflating: bert_base_th/model.ckpt.meta  
  inflating: bert_base_th/bert_config.json  
  inflating: bert_base_th/model.ckpt.data-00000-of-00001  
Archive:  th_wiki_bpe.zip
  inflating: th.wiki.bpe.op25000.model  
  inflating: th.wiki.bpe.op25000.vocab  


# Load extra library

In [ ]:
# cut sub word with sentencepiece
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 5.3MB/s 


# Make Output Directory

In [ ]:
!mkdir output

# Fine tuning (20 minutes running on GPU)

In [ ]:
import os
os.environ['BPE_DIR'] = "/content"
os.environ['WONGNAI_DIR'] = "/content/wongnai-corpus/review"
os.environ['OUTPUT_DIR'] = "/content/output"
os.environ['BERT_BASE_DIR'] = "/content/bert_base_th"

In [ ]:
# train with GPU
!python bert/run_classifier.py \
  --task_name=wongnai \
  --do_train=true \
  --do_predict=true \
  --data_dir=$WONGNAI_DIR \
  --vocab_file=$BPE_DIR/th.wiki.bpe.op25000.vocab \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --learning_rate=5e-5 \
  --num_train_epochs=2.0 \
  --output_dir=$OUTPUT_DIR \
  --spm_file=$BPE_DIR/th.wiki.bpe.op25000.model

Streaming output truncated to the last 5000 lines.
I1025 08:49:01.861835 140155373832064 tpu_estimator.py:2308] examples/sec: 35.0087
INFO:tensorflow:global_step/sec: 1.09546
I1025 08:49:02.774283 140155373832064 tpu_estimator.py:2307] global_step/sec: 1.09546
INFO:tensorflow:examples/sec: 35.0546
I1025 08:49:02.774611 140155373832064 tpu_estimator.py:2308] examples/sec: 35.0546
INFO:tensorflow:global_step/sec: 1.10275
I1025 08:49:03.681149 140155373832064 tpu_estimator.py:2307] global_step/sec: 1.10275
INFO:tensorflow:examples/sec: 35.2881
I1025 08:49:03.681376 140155373832064 tpu_estimator.py:2308] examples/sec: 35.2881
INFO:tensorflow:global_step/sec: 1.10241
I1025 08:49:04.588241 140155373832064 tpu_estimator.py:2307] global_step/sec: 1.10241
INFO:tensorflow:examples/sec: 35.2771
I1025 08:49:04.588466 140155373832064 tpu_estimator.py:2308] examples/sec: 35.2771
INFO:tensorflow:global_step/sec: 1.11277
I1025 08:49:05.486912 140155373832064 tpu_estimator.py:2307] global_step/sec: 1.1

# Save the output folder

In [ ]:
!mkdir output_last

In [ ]:
!cp output/model.ckpt-2500.* output_last/

cp: cannot stat 'output/model.ckpt-2500.*': No such file or directory


In [ ]:
%cd output_last

/content/output_last


In [ ]:
#Remove the epoch postfix
!mv model.ckpt-2500.data-00000-of-00001 model.ckpt.data-00000-of-00001
!mv model.ckpt-2500.index model.ckpt.index
!mv model.ckpt-2500.meta model.ckpt.meta

mv: cannot stat 'model.ckpt-2500.data-00000-of-00001': No such file or directory
mv: cannot stat 'model.ckpt-2500.index': No such file or directory
mv: cannot stat 'model.ckpt-2500.meta': No such file or directory


In [ ]:
!ls

bert		  output	   sample_data		      th_wiki_bpe.zip
bert_base_th	  output_last	   th.wiki.bpe.op25000.model  wongnai-corpus
bert_base_th.zip  output_last.zip  th.wiki.bpe.op25000.vocab


In [ ]:
%cd output

/content/output


In [ ]:
!cat test_results.tsv

0.23052715	0.26463053	0.102493264	0.25600362	0.14634547
0.22588374	0.23827775	0.13355415	0.2301219	0.17216241
0.21679117	0.27434492	0.1285982	0.23735395	0.1429118
0.18600091	0.2717704	0.1318395	0.26516882	0.14522044
0.24731834	0.28293297	0.123363964	0.20074141	0.14564341
0.24646556	0.26712474	0.122590914	0.19284913	0.17096962
0.23091248	0.25818405	0.12787671	0.21279648	0.17023031
0.26025492	0.23678413	0.11449464	0.20647575	0.18199053
0.22736776	0.25908473	0.12857513	0.21118835	0.17378405
0.24437891	0.28872946	0.12621008	0.1966876	0.14399391
0.22911851	0.27672833	0.14340225	0.17393604	0.17681493
0.24669708	0.27849686	0.11449161	0.22846907	0.13184533
0.18380201	0.30742365	0.13238308	0.22485259	0.15153867
0.2592926	0.22219536	0.13437957	0.2243298	0.15980265
0.23218237	0.26166132	0.109352075	0.25339785	0.14340633
0.2229893	0.2525148	0.12859865	0.23673329	0.15916401
0.19333075	0.28902072	0.116458364	0.27867478	0.12251536
0.2658515	0.28754976	0.099353254	0.20076038	0.14648515
0.24338034	0.25

In [ ]:
% cd ..

/content


In [ ]:
!zip -r output_last.zip output_last

updating: output_last/ (stored 0%)


In [ ]:
from google.colab import files
files.download('output_last.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>